In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.6f}'.format)

from astropy import units as u
from astropy.coordinates import Angle, SkyCoord, get_constellation, EarthLocation, AltAz, get_sun
from astroquery.vizier import Vizier

from astropy.time import Time, TimeDelta
from datetime import datetime, timedelta
import pytz
import requests

In [2]:
spp_data = ['SW AND',
            'XX AND', 
            'AT AND', 
            'SW AQR', 
            'TZ AUR', 
            'BH AUR', 
            'RS BOO', 
            'ST BOO', 
            'SW BOO', 
            'SZ BOO', 
            'TV BOO', 
            'TW BOO', 
            'UU BOO', 
            'UY BOO', 
            'RW CNC', 
            'TT CNC', 
            'RR CET', 
            'XZ CYG', 
            'DM CYG', 
            'RW DRA', 
            'XZ DRA', 
            'RR GEM', 
            'VX HER',
            'TV BOO', 
            'TW BOO', 
            'UU BOO', 
            'UY BOO', 
            'RW CNC', 
            'TT CNC', 
            'RR CET', 
            'XZ CYG', 
            'DM CYG', 
            'RW DRA', 
            'XZ DRA', 
            'RR GEM', 
            'VX HER',
            'AR HER', 
            'DL HER', 
            'DY HER', 
            'SZ HYA',
            'UU HYA', 
            'DG HYA', 
            'DH HYA', 
            'RR LEO', 
            'SS LEO', 
            'TV LEO',
            'WW LEO',
            'SZ LYN',
            'RR LYR', 
            'RZ LYR', 
            'AV PEG', 
            'RV UMA']
                      
df_spp = pd.DataFrame(spp_data, columns=['displayName'])   
df_spp['variableType'] = 'SPP'

eb_data = ['W UMA']
df_eb = pd.DataFrame(eb_data, columns=['displayName'])   
df_eb['variableType'] = 'EB'

df = pd.concat([df_spp, df_eb], ignore_index=True)

In [3]:
# Function to fetch data from the VSX API and update the DataFrame
def fetch_vsx_data(row):
    displayName = row['displayName']
    url = f'https://www.aavso.org/vsx/index.php?view=api.object&ident={displayName}&format=json'
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        json_data = response.json()

        # Check if the response contains 'VSXObject'
        if 'VSXObject' in json_data:
            vsx_object = json_data['VSXObject']
            
            row['displayName'] = vsx_object.get("Name", displayName)
            
            row['OID'] = vsx_object.get("OID", None)
            
            ra = Angle(vsx_object.get('RA2000', None), u.deg).to(u.hourangle).value
            row['ra'] = ra
            
            dec = Angle(vsx_object.get('Declination2000', None), u.deg).value
            row['dec'] = dec
            
            coord = SkyCoord(ra=ra * u.deg, dec=dec * u.deg)
            row['constellation'] = get_constellation(coord)
                                  

            row['period'] = vsx_object.get('Period', pd.NA)
            row['epoch'] = vsx_object.get('Epoch', pd.NA)
            row['magnitudeHigh'] = vsx_object.get('MaxMag', pd.NA)
            row['magnitudeLow'] = vsx_object.get('MinMag', pd.NA)
            row['riseDuration'] = vsx_object.get('RiseDuration', pd.NA)                   
            row['type'] = vsx_object.get('VariabilityType', pd.NA)
            row['spectrualType'] = vsx_object.get('SpectralType', pd.NA)
        else:
            print(f"No data found for {displayName}")
    
    except requests.exceptions.RequestException as e:
        print(f"Request failed for {displayName}: {e}")
    
    return row

In [4]:
# Apply the function to each row in the DataFrame
df = df.apply(fetch_vsx_data, axis=1)
df[0:200]

,displayName,variableType,OID,ra,dec,constellation,period,epoch,magnitudeHigh,magnitudeLow,riseDuration,type,spectrualType
0,SW And,SPP,23,0.395303,29.401000,Pegasus,0.442262,2454346.75,9.14 V,10.14 V,17,RRAB/BL:,A7III-F8III
1,XX And,SPP,49,1.290947,38.950560,Andromeda,0.722757,2453662.575,10.08 V,11.13 V,19,RRAB/BL,A8-F6
2,AT And,SPP,73,23.708564,43.014360,Andromeda,0.6169144,2453287.4851,10.42 V,10.92 V,21,RRAB,F0-F7
3,SW Aqr,SPP,868,21.254959,0.076360,Cetus,0.4593047,2425097.3771,10.438 V,11.755 V,11,RRAB,A5-G1
4,TZ Aur,SPP,3795,7.193061,40.776970,Andromeda,0.391674615,2419902.4324,11.08 V,12.45 V,14,RRAB,A9-F6
5,BH Aur,SPP,3848,5.201186,33.963060,Andromeda,0.4560900,2442751.642,11.22 V,12.20 V,16,RRAB,<NA>
6,RS Boo,SPP,4314,14.559225,31.754610,Pisces,0.3773369,2460458.551,9.69 V,10.84 V,17,RRAB/BL,A7-F5
7,ST Boo,SPP,4323,15.510897,35.784530,Andromeda,0.622290687,2419181.486,10.49 V,11.41 V,19,RRAB/BL,A7-F7
8,SW Boo,SPP,4326,14.459683,36.045580,Andromeda,0.5135281,2442570.76,11.76 V,12.88 V,13,RRAB,F6
9,SZ Boo,SPP,4329,14.703714,28.206560,Pisces,0.5228184,2453471.3551,11.92 V,13.05 V,12,RRAB,A3-F6


In [5]:
df.to_json('variable_stars.json', orient='records', lines=False)